# Run your AOAI app as an Azure Function App 

In this workshop you will deploy a simple app submitting a prompt to your AOAI endpoints.
You will 
-  create your dev environment 
- create your Azure Function App
- test your app locally 
- deploy your app to Azure Function App
- test your production deployment

In [ ]:
az login --use-device-code
#will give you a device code. Go to microsoft.com/devicelogin from 
#any other browser and put in the device-code to log in. 


#Create and activate a virtual environment 
sudo apt-get install python3-venv
python -m venv .venv2
source .venv2/bin/activate

#upgrade python to 3.9
sudo apt-get update
sudo apt-get upgrade
sudo apt-get install -y build-essential zlib1g-dev libncurses5-dev libgdbm-dev libnss3-dev libssl-dev libsqlite3-dev libreadline-dev libffi-dev curl libbz2-dev

cd /opt
sudo wget https://www.python.org/ftp/python/3.9.0/Python-3.9.0.tgz
sudo tar -xvf Python-3.9.0.tgz
cd Python-3.9.0
sudo ./configure --enable-optimizations
sudo make altinstall

python3.9 --version

#Create a new venv with Python3.9
python3.9 -m venv my-afa
source my-afa/bin/activate
sudo npm install -g azure-functions-core-tools@4 --unsafe-perm true
npm install -g npm@9.8.0


### Locally develop & test your Azure Function app

In [ ]:
#Create a local project
func init aoai-app-02 --python -m V2 
cd aoai-app-02

#ssh into the machine (add NSG permit ssh lines to VM VNET on Azure portal)
#launch VS Code leveraging VSCore remote connection plugin
code .

#edit requirements.txt and add openai 
#edit function_app.py and add the below...

#Create your AOAI app 

source my-afa/bin/activate
sudo apt install python3-pip
pip3 install -r requirements.txt

### Write your function app 
Add the below code to function_app.py

In [ ]:
import azure.functions as func
import openai 

app = func.FunctionApp()

@app.function_name(name="HttpTrigger1")
@app.route(route="hello")
def test_function(req: func.HttpRequest) -> func.HttpResponse:
    
    openai.api_type="azure"
    openai.api_version="2023-05-15"

    OPENAI_API_BASE = "XXX"
    OPENAI_API_KEY = "XXX"

    openai.api_base = OPENAI_API_BASE
    openai.api_key = OPENAI_API_KEY
    
    prompt = req.params.get('prompt')
    
    if not prompt:
        return func.HttpResponse(
            "Please provide a prompt.",
            status_code=400
        )        
    completions = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    message = completions.choices[0].message.content.strip()
    return func.HttpResponse(message)

### Test Locally 

In [ ]:
#install upgrade node/npm
sudo apt install npm
sudo npm cache clean -f
sudo npm install -g n
sudo n stable
node -v 
npm -v 


#install 
sudo npm install -g azurite
azureit 

#on another terminal and from within aoai-app-02 directory 
func start 

#test yet on another terminal with...
curl http://localhost:7071/api/hello?prompt=explain%20quantum%20physics

### Setup Azure resources required to deploy the Azure Function App

In [ ]:
z login --use-device-code

#create resource-group 
az group create --name rg-openai --location westeurope

#create a storage account 
az storage account create --name saopenaiozguler --location westeurope --resource-group rg-openai --sku Standard_LRS

#create the function app in Azure
#Go to aoai-app-01 folder and run the below within the folder...
sudo apt-get upgrade azure-cli
az functionapp create --resource-group rg-openai --consumption-plan-location westeurope --runtime python --runtime-version 3.9 --functions-version 4 --name aoai-app-01 --os-type linux --storage-account saopenaiozguler2

### Deploy your app

In [ ]:
#Create your function container
az functionapp create --resource-group rg-test-app-xxx  --consumption-plan-location eastus --runtime python --runtime-version 3.9 --functions-version 4 --name test-app-xxx --os-type linux --storage-account storageappxxxozguler

#Deploy your app 
func azure functionapp publish aoai-app-02 

#For observability add the following...
func azure functionapp logstream aoai-app-02 --browser

(If the “function create” command fails above -it kept failing for me, possibly a CLI bug - you can create the functionapp through VSCode Function App plugin or via Azure Portal and skip the function app create CLI step above and move to the functionapp publish step. Although I used the CLI command MS Learn documentation, I got “ — functions-version 4” after wasting enough time troubleshooting this I moved and simply created the function on Azure portal. Below is the error message I got…)

### Test your app 

To test your Azure Function app with Postman,
Change the HTTP method to POST
Add header “content-type” application/json
Add x-functions-key and add your functions master key, you will find under “keys” in the portal…
Add body raw text {}